In [ ]:
using ReactionMechanismSimulator
using PythonPlot
using DifferentialEquations
using Sundials

In [ ]:
phaseDict = readinput("../src/testing/ch4o2cat.rms") #load mechanism dictionary

In [ ]:
gasspcs = phaseDict["gas"]["Species"]; #mechanism dictionaries index:  phaseDict[phasename]["Species" or "Reactions"]
gasrxns = phaseDict["gas"]["Reactions"];
surfacespcs = phaseDict["surface"]["Species"]
surfacerxns = phaseDict["surface"]["Reactions"]
interfacerxns = phaseDict[Set(["gas","surface"])]["Reactions"];

In [ ]:
ig = IdealGas(gasspcs,gasrxns;name="gas"); #Define the phase (how species thermodynamic and kinetic properties calculated)
cat = IdealSurface(surfacespcs,surfacerxns,2.486e-5;name="surface");

In [ ]:
initialconds = Dict(["T"=>800.0,"P"=>1.0e5,
        "O2"=>0.2,
        "N2"=>0.7,
        "CH4"=>0.1]); #Set simulation Initial Temp and Pressure
domaingas,y0gas,pgas = ConstantTPDomain(phase=ig,initialconds=initialconds,); #Define the domain (encodes how system thermodynamic properties calculated)

In [ ]:
V = 8.314*800.0/1.0e5
A = 1.0e5*V
initialconds = Dict(["T"=>800.0,"A"=>A,"vacantX"=>cat.sitedensity*A]); #Set simulation Initial Temp and Pressure
domaincat,y0cat,pcat = ConstantTAPhiDomain(phase=cat,initialconds=initialconds,); #Define the domain (encodes how system thermodynamic properties calculated)

In [ ]:
inter,pinter = ReactiveInternalInterfaceConstantTPhi(domaingas,domaincat,interfacerxns,800.0,A);

In [ ]:
react,y0,p = Reactor((domaingas,domaincat),(y0gas,y0cat),(0.0,0.1),(inter,),(pgas,pcat,pinter)); #Create the reactor object

In [ ]:
@time sol = solve(react.ode,CVODE_BDF(),abstol=1e-20,reltol=1e-6); #solve the ode associated with the reactor

In [ ]:
sol.retcode == :Success

In [ ]:
ssys = SystemSimulation(sol,(domaingas,domaincat,),(inter,),p);

In [ ]:
plotmolefractions(ssys.sims[2];exclude=["N2"],tol=0.001)
xlim(0.0,3e-5)

In [ ]:
plotmolefractions(ssys.sims[1];exclude=["N2"])
xlim(0.0,3e-5)

In [ ]:
plotrops(ssys,"H2OX",0.2e-5)

In [ ]:
plotrops(ssys,"CO2X",1e-5)

In [ ]:
getfluxdiagram(ssys,0.1e-5)